In [9]:
%cd ..

c:\Users\kuhnie\code-chicago\gcfd\code-for-chicago-gcfd


In [11]:
from src.census_response import CensusData

In [12]:
CensusData.load_df()

In [19]:
zip_df = CensusData.df_dict['zip']
zip_df.head()

,race_total,race_native,race_black,race_white,race_twoplus_total,race_pacific,race_other,race_asian,race_hispaniclatino_total,poverty_population_total,poverty_population_poverty,poverty_population_poverty_child,race_percentages,race_majority,poverty_percentages
60002,24066,4,525,20862,669,0,0,344,1662,24014,1809,475,"{'race_native': 0.000166, 'race_black': 0.0218...",race_white,"{'poverty_population_poverty': 0.075331, 'pove..."
60004,50915,29,411,40720,740,0,0,5002,4013,50372,2409,662,"{'race_native': 0.00057, 'race_black': 0.00807...",race_white,"{'poverty_population_poverty': 0.047824, 'pove..."
60005,29539,0,667,22700,245,0,59,2705,3163,29314,2317,475,"{'race_native': 0.0, 'race_black': 0.02258, 'r...",race_white,"{'poverty_population_poverty': 0.079041, 'pove..."
60007,33420,17,387,25084,512,0,35,3365,4020,33201,1271,234,"{'race_native': 0.000509, 'race_black': 0.0115...",race_white,"{'poverty_population_poverty': 0.038282, 'pove..."
60008,21833,0,1009,13178,348,92,0,1776,5430,21668,1362,325,"{'race_native': 0.0, 'race_black': 0.046214, '...",race_white,"{'poverty_population_poverty': 0.062858, 'pove..."


In [20]:
county_df = CensusData.df_dict['county']
county_df.head()

,NAME,race_total,race_native,race_black,race_white,race_twoplus_total,race_pacific,race_other,race_asian,race_hispaniclatino_total,poverty_population_total,poverty_population_poverty,poverty_population_poverty_child,race_percentages,race_majority,poverty_percentages
17001,"Adams County, Illinois",66427,178,2676,61009,889,42,72,540,1021,64844,7874,2385,"{'race_native': 0.00268, 'race_black': 0.04028...",race_white,"{'poverty_population_poverty': 0.12143, 'pover..."
17003,"Alexander County, Illinois",6532,48,2169,4063,173,0,0,6,73,6440,1946,813,"{'race_native': 0.007348, 'race_black': 0.3320...",race_white,"{'poverty_population_poverty': 0.302174, 'pove..."
17005,"Bond County, Illinois",16712,50,1043,14705,118,0,22,196,578,14965,2168,602,"{'race_native': 0.002992, 'race_black': 0.0624...",race_white,"{'poverty_population_poverty': 0.144871, 'pove..."
17007,"Boone County, Illinois",53606,153,1084,39370,848,0,59,633,11459,53089,5843,2222,"{'race_native': 0.002854, 'race_black': 0.0202...",race_white,"{'poverty_population_poverty': 0.11006, 'pover..."
17009,"Brown County, Illinois",6675,6,1343,4933,66,0,4,37,286,4804,482,132,"{'race_native': 0.000899, 'race_black': 0.2011...",race_white,"{'poverty_population_poverty': 0.100333, 'pove..."
